In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import chainer
import numpy as np
from PIL import Image
import chainer.functions as F

In [ ]:
from net import Generator
gen = Generator(100)

In [ ]:
chainer.serializers.load_npz('../data/weights/gen_iter_190000.npz', gen)

In [ ]:
# check output
x = gen.make_hidden(1)
with chainer.using_config('train', False), chainer.using_config('enable_backprop', False):
    y = gen(x)
    
y = y[0].array
Image.fromarray((y*255).astype(np.uint8).transpose(1,2,0))

In [ ]:
import onnx_chainer

In [ ]:
from net import Generator
gen = Generator(100)

In [ ]:
import numpy as np
x = np.zeros(100, dtype=np.float32).reshape(1, -1)

In [ ]:
# modify trained model to output 4channel(RGBA) and its vaues in [0, 255], order is (batchsize, h, w, ch)
class FillAlpha(chainer.Chain):
    def __init__(self, w, h):
        super().__init__()
        self.x = chainer.Variable(np.ones((1, 1, h, w), dtype=np.float32))
        
    def forward(self, x):
        return F.concat([x, self.x], axis=1)

model = chainer.Sequential(gen, FillAlpha(32, 32), lambda x: x*255, lambda x: x.transpose(0, 2, 3, 1))

In [ ]:
model(np.zeros((1, 100), dtype=np.float32)).shape

In [ ]:
onnx_chainer.export(model, np.zeros((1, 100), dtype=np.float32), '../web/gen.onnx')

In [ ]:
from PIL import Image
Image.fromarray(model(gen.make_hidden(1)).array.astype(np.uint8)[0])